# Skriptit

Python skripti on Pythonilla kirjoitettu ohjelma, joka on tallennettu .py-muodossa. Usein Python-ohjelmat tallennetaan .py-muodossa (poikkeuksena Jupyter-muistiot).

Kun skripti kirjoitetaan Jupyter notebookilla, niin kaikki komennot kirjoitetaan samaan soluun ja viedään .py-muotoon valitsemalla **File – Save and Export Notebook as – Executable Script**. Skriptejä on kuitenkin kätevämpää kirjoittaa ja testata ohjelmalla *Visual Studio Code*. 

Python skripti voidaan käynnistää suoraan komentoriviltä. Jos on asentanut materiaalipaketin (tässä pyex-kansio) suoraan kotikansion alle, niin voi ensin siirtyä pyex-kansioon komennolla **cd pyex**  ja käynnistää esimerkiksi co2.py-skriptin komentorivin komennolla **python co2.py**. Skripti hakee ilmakehän kuukausittaiset hiilidioksidipitoisuudet Global Monitoring laboratoryn sivuilta, esittää ne viivakaaviona ja ennustaa hiilidioksidipitoisuudet 36 kuukautta eteenpäin.

In [ ]:
### Skriptin co2.py koodi

# Tuoreimman datan nouto ja siivous
# Ennuste 36 kuukautta eteenpäin
# Grafiikka, jossa aikasarja ja liukuva 12 kuukauden keskiarvo
# Grafiikka, jossa ennuste 36 kuukautta eteenpäin
# Data, ennusteet ja grafiikat Exceliin

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xlwings as xw
from statsmodels.tsa.api import ExponentialSmoothing
import warnings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

# Datan nouto ja siivous
df = pd.read_csv('https://www.esrl.noaa.gov/gmd/webdata/ccgg/trends/co2/co2_mm_mlo.txt',\
sep='\s+', skiprows=58, usecols=[0, 1, 3], names=['year', 'month', 'average'])
df.index = pd.to_datetime(df['year'].astype(str) + df['month'].astype(str), format='%Y%m')
df = df.drop(['year', 'month'], axis=1)

# Ennustemalli kolminkertaista eksponentiaalista tasoitusta käyttäen
malli = ExponentialSmoothing(df['average'], trend='add', seasonal='mul', seasonal_periods=12, freq='MS').fit()

# Ennusteet 36 kuukautta eteenpäin
next_date = df.index.to_series().iloc[-1]
if next_date.month == 12:
    index = pd.date_range(f'{next_date.year}-{1}-{next_date.day}', periods=36, freq='MS')
else:
    index = pd.date_range(f'{next_date.year}-{next_date.month+1}-{next_date.day}', periods=36, freq='MS')
ennusteet = malli.forecast(36)
df_ennuste = pd.DataFrame(data=ennusteet, index=index, columns=['Ennuste'])

# Grafiikka
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(8, 8), tight_layout=True)
df.plot(ax=ax[0], title='Hiilidioksidipitoisuudet ja 12 kuukauden liukuva keskiarvo', legend=False)
df.rolling(12).mean().plot(ax=ax[0], legend=False)
df['2000':]['average'].plot(ax=ax[1], title = 'Ennuste 3 vuotta eteenpäin')
df_ennuste['Ennuste'].plot(ax=ax[1])

# Excel-työkirjan alustus
app = xw.App()
wb = app.books[0]
sht = wb.sheets[0]
sht.name = 'co2'

# Data ja ennusteet Exceliin
xw.Range('A1:H1').add_hyperlink(address='https://www.esrl.noaa.gov/gmd/webdata/ccgg/trends/co2/co2_mm_mlo.txt', \
    text_to_display='Lähde: https://www.esrl.noaa.gov/gmd/webdata/ccgg/trends/co2/co2_mm_mlo.txt')
xw.Range('A3').value = df
xw.Range('B3').value = 'Monthly average ppm'
xw.Range('D43').value = df_ennuste

# Grafiikka Exceliin
sht.pictures.add(fig, anchor=xw.Range('D3'))

Skripti voi ottaa vastaan myös parametreja. Materiaalipaketin skripti osake.py odottaa osakkeen Yahoo Finance -tunnusta. Sen voi antaa skriptiä kutsuttaessa, esimerkiksi **python osake.py NOKIA.HE**. Jos parametria ei anneta kutsussa, niin skripti osaa kysyä sitä. Tutustu skriptiin: tunnistatko sieltä komennot, jotka liittyvät parametrin lukemiseen ja kysymiseen?

In [ ]:
### Skriptin osake.py koodi

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xlwings as xw
import yfinance as yf
import sys
sns.set_style('whitegrid')

if (len(sys.argv) > 1):
    x = sys.argv[1]
else:
    x = input('Anna osakkeen Yahoo Finance tunnus: ')

try:
    stock = yf.download(x, start='2023-1-1')
except:
    print('Tunnuksella ei löytynyt tietoja')
    exit()

omxh25 = yf.download('^OMXH25', start='2023-1-1')
vertailu = pd.concat([stock['Close'].pct_change(), omxh25['Close'].pct_change()], axis=1)
vertailu.columns = [x, 'OMXH25']
fig = plt.figure()
plt.scatter(x=vertailu['OMXH25'], y=vertailu[x])
plt.xlabel('OMXH25 päivämuutos')
plt.ylabel(f'{x} päivämuutos')

app = xw.App(visible=False)
wb = xw.books.active
ws1 = wb.sheets[0]
ws1.name = x
ws2 = wb.sheets.add(name='omxh25')
ws3 = wb.sheets.add(name='vertailu')
ws1.range('A1').value = stock
ws2.range('A1').value = omxh25
ws3.range('A1').value = vertailu
ws3.pictures.add(fig, anchor=xw.Range('E1'))
app.visible = True

Materiaalipaketin skripti **pika.py** laskee annetulle datalle frekvenssitaulukot, ristiintaulukoinnit ja tilastolliset tunnusluvut. Testaa skriptiä esimerkiksi titanic-datalle: **python pika.py data/titanic.xlsx**.

Materiaalipaketin skripti **bollinger.py** hakee annetun osakkeen kurssihistorian ja piirtää sen Bollingerin nauhat. Bollingerin nauhat ovat arvopaperimarkkinoilla käytetyn teknisen analyysin mittareita, joiden avulla pyritään tunnistamaan yliostettuja ja ylimyytyjä tilanteita sekä markkinaheilunnan muutoksia. Lisätietoa Bollingerin nauhoista löytyy esimerkiksi artikkeleista https://www.investopedia.com/terms/b/bollingerbands.asp (Investopedia) ja https://www.sijoittaja.fi/276210/tekninen-analyysi/ (Sijoittaja).

Kokeile esimerkiksi **python bollinger.py TELIA1.HE**.

In [ ]:
### Skriptin bollinger.py koodi

import pandas as pd
import matplotlib.pyplot as plt
import xlwings as xw
import yfinance as yf
import sys

def bollinger(stock, span=20):
    
    '''Piirtää Bollingerin nauhat'''
    
    stock['MA'] = stock['Close'].rolling(span).mean()
    stock['STD'] = stock['Close'].rolling(span).std(ddof = 0) 
    stock['Upper'] = stock['MA'] + (stock['STD'] * 2)
    stock['Lower'] = stock['MA'] - (stock['STD'] * 2)

    fig, ax = plt.subplots()
    stock['Close'].plot(label='Close', color='black', figsize=(10, 6))
    stock['Upper'].plot(label='Upper', linestyle='--', linewidth=1, color='red')
    stock['MA'].plot(label='Middle', linestyle='--', linewidth=1.2, color='grey')
    stock['Lower'].plot(label='Lower', linestyle='--', linewidth=1, color='red')
    plt.gca().fill_between(stock.index, stock['Lower'], stock['Upper'], facecolor='yellow', alpha=0.1)
    plt.legend()
    plt.title('BOLLINGER BANDS')

    return stock, fig


if (len(sys.argv) > 1):
    tunnus = sys.argv[1]
else:
    tunnus = input('Anna osakkeen Yahoo Finance tunnus: ')

try:
    stock = yf.download(tunnus, start='2022-1-1')
except:
    print('Tunnuksella ei löytynyt tietoja')
    exit()
    
stock, fig = bollinger(stock)
app = xw.App()
wb = app.books[0]
ws = wb.sheets[0]
ws.name = 'bollinger'
xw.Range('A1').value = tunnus
xw.Range('A30').value = stock
ws.pictures.add(fig, anchor = xw.Range('A3'))
wb.activate()


Materiaalipaketin skripti **rsi.py** hakee annetun osakkeen kurssihistorian ja piirtää RSI:n. Relative Strength Index eli suhteellinen voimaindeksi on myös eräs monista arvopaperimarkkinoilla käytetyn teknisen analyysin mittari. Se mittaa osakkeen tai muun arvopaperin hinnan muutoksen suuntaa ja voimakkuutta viimeaikaisiin hintoihin pohjautuen ja pyrkii arvioimaan, kuinka yli- tai aliostettuja kyseiset arvopaperit ovat. Lisätietoa RSI:stä esimerkiksi https://www.investopedia.com/terms/r/rsi.asp (Investopedia) ja https://www.sijoittaja.fi/365376/teknisen-analyysin-indikaattorit-relative-strenght-index/ ja https://www.sijoittaja.fi/276210/tekninen-analyysi/ (Sijoittaja).

Kokeile esimerkiksi **python rsi.py ELISA.HE**.

In [ ]:
### Skriptin rsi.py koodi

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xlwings as xw
import yfinance as yf
import sys
sns.set_style('whitegrid')


def rsi(stock, span=14):
    
    muutos = stock['Close'].diff()
    
    u = muutos.clip(lower = 0) # positiiviset muutokset
    d = muutos.clip(upper = 0).abs() # negatiiviset muutokset
    
    avgu = u.rolling(span).mean() # positiivisten muutosten liukuva keskiarvo
    avgd = d.rolling(span).mean() # negatiivisten muutosten liukuva keskiarvo

    rs = avgu/avgd # suhteellinen voima (relative strength)

    stock['RSI'] = (100 - (100/(1 + rs))) # suhteellinen voimaindeksi (RSI)

    # Kaavio
    fig, ax = plt.subplots()
    stock['RSI'].plot(figsize = (10, 6), label = 'yksinkertainen liukuva', legend = True)
    plt.ylim(0, 100)
    plt.axhline(30, color = 'r', linestyle = '--')
    plt.axhline(70, color = 'r', linestyle = '--')
    plt.ylabel('RSI')
    
    return stock, fig

if (len(sys.argv) > 1):
    tunnus = sys.argv[1]
else:
    tunnus = input('Anna osakkeen Yahoo Finance tunnus: ')

try:
    data = yf.download(tunnus, start = '2022-1-1')
except:
    print('Tunnuksella ei löytynyt tietoja')
    exit()
    
stock, fig = rsi(data)

wb = xw.Book()
ws = wb.sheets[0]
ws.name = 'rsi'
xw.Range('A1').value = tunnus
xw.Range('A30').value = stock
ws.pictures.add(fig, anchor = xw.Range('A3'))
wb.activate()


<u>Lähde ja lisämateriaalia</u>: 
Aki Taanila: Data-analytiikka Pythonilla, https://tilastoapu.wordpress.com/python/.

In [1]:
import datetime
print(f'Last modified {datetime.datetime.now():%Y-%m-%d %H:%M} by Juha Nurmonen')

Last modified 2024-04-06 15:45 by Juha Nurmonen
